In [9]:
!pip install rank-bm25


In [13]:
pip install sentencepiece


   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------------------------------------- 10.2/992.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/992.0 kB 435.7 kB/s eta 0:00:03
   ------ --------------------------------- 153.6/992.0 kB 1.1 MB/s eta 0:00:01
   ------------------ --------------------- 450.6/992.0 kB 2.8 MB/s eta 0:00:01
   --------------------------------- ------ 829.4/992.0 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 992.0/992.0 kB 4.2 MB/s eta 0:00:00


In [3]:
import json
import re
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the JSON file with specified encoding
file_path = 'news.article.json'

with open(file_path, 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Preprocess articles
def clean_article(article):
    # Remove special characters and noise
    article = re.sub(r'\s+', ' ', article)  # Remove extra whitespace
    article = re.sub(r'[^\w\s]', '', article)  # Remove punctuation
    article = article.lower()  # Convert to lowercase
    return article

cleaned_articles = []
for article in tqdm(articles):
    article_text = article.get('articleBody', '')  # Access the article text using 'articleBody' key
    if 'israel' in article_text.lower() or 'hamas' in article_text.lower():  # Basic filter for relevant articles
        cleaned_articles.append(clean_article(article_text))

# Tokenize the articles for BM25
tokenized_articles = [article.split() for article in cleaned_articles]

# Initialize BM25
bm25 = BM25Okapi(tokenized_articles)

def retrieve_articles(query, k=5):
    tokenized_query = query.lower().split()
    results = bm25.get_top_n(tokenized_query, cleaned_articles, n=k)
    return results

# Load T5 model and tokenizer
model_name = 't5-small'  # You can choose a larger model for better performance
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def answer_question(question, articles):
    context = " ".join(articles)
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt', truncation=True)
    outputs = model.generate(input_ids)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def answer_israel_hamas_question(question):
    retrieved_articles = retrieve_articles(question)
    answer = answer_question(question, retrieved_articles)
    return answer

# Repeatedly ask for questions and provide answers
while True:
    user_question = input("Please enter your question (or type 'exit' to stop): ")
    if user_question.lower() == 'exit':
        print("Exiting the question-answering system. Goodbye!")
        break
    answer = answer_israel_hamas_question(user_question)
    print("Answer:", answer)


100%|███████████████████████████████████████████████████████████████████████████| 37421/37421 [03:47<00:00, 164.51it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Please enter your question (or type 'exit' to stop):  What is happening in the hospital


Answer: protests took place in parts of tel aviv after israeli military announced that


Please enter your question (or type 'exit' to stop):  Exit


Exiting the question-answering system. Goodbye!
